In [3]:
# !pip install catboost

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import pyarrow.parquet as pq

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,slctn_nmbr,client_id,npo_account_id,npo_accnts_nmbr,pmnts_type,year,quarter,gender,age,clnt_cprtn_time_d,...,lk,assignee_npo,assignee_ops,postal_code,region,citizen,fact_addrss,appl_mrkr,evry_qrtr_pmnt,churn
0,1,0xA095932790098744A2325A8D152C05C7,0xD4DBBAC6561929409BA45725A220613E,1,2,2017,2017Q4,1,48,8091,...,-1,-1,-1,446254.0,САМАРСКАЯ ОБЛ,-1,-1,0,0,0
1,0,0xF75BAA20A603A2479005DEB80464336E,0x3DCFF11974B3D8458CE3569DE9E855D5,2,2,2009,2009Q1,1,35,3303,...,-1,-1,-1,399841.0,ЛИПЕЦКАЯ ОБЛ,-1,1,0,1,0
2,2,0x0D18081E0556AF4F95A931FBA4159D52,0x9DADF88CB3407C4E89403315F640393E,1,1,2010,2010Q1,1,47,1163,...,-1,-1,-1,302038.0,ОРЛОВСКАЯ ОБЛ,1,1,0,1,0
3,0,0x0AD72BC5475D9A4D83E12350A84F839D,0xF59C4E62F115684899EAE9B5D5ECF70F,1,1,2005,2005Q4,-1,42,47,...,-1,-1,-1,620034.0,СВЕРДЛОВСКАЯ ОБЛ,1,1,0,1,0
4,0,0x73DF06339CC25840838408BAD52BE223,0x35BFFA05321C014B9FC29A189E89EDF0,2,2,2007,2007Q4,1,42,876,...,-1,-1,-1,398035.0,ЛИПЕЦКАЯ ОБЛ,1,1,0,1,0


In [4]:
df.shape

(3589918, 42)

In [5]:
df.isna().sum()

slctn_nmbr                      0
client_id                       0
npo_account_id                  0
npo_accnts_nmbr                 0
pmnts_type                      0
year                            0
quarter                         0
gender                          0
age                             0
clnt_cprtn_time_d               0
actv_prd_d                      0
lst_pmnt_rcnc_d                 0
balance                         0
oprtn_sum_per_qrtr              0
oprtn_sum_per_year              0
frst_pmnt_date              32441
lst_pmnt_date_per_qrtr    1156410
frst_pmnt                       0
lst_pmnt                        0
pmnts_sum                       0
pmnts_nmbr                      0
pmnts_sum_per_qrtr              0
pmnts_sum_per_year              0
pmnts_nmbr_per_qrtr             0
pmnts_nmbr_per_year             0
incm_sum                        0
incm_per_qrtr                   0
incm_per_year                   0
mgd_accum_period                0
mgd_payment_pe

In [6]:
df.duplicated().sum()

1947

In [7]:
df.drop_duplicates(inplace = True)

In [8]:
column_types = df.dtypes

# Отфильтровать столбцы с типом 'object', которые обычно являются категориальными
categorical_columns = column_types[column_types == 'object'].index.tolist()

In [9]:
categorical_columns

['client_id',
 'npo_account_id',
 'quarter',
 'frst_pmnt_date',
 'lst_pmnt_date_per_qrtr',
 'region']

In [10]:
for i in categorical_columns:
    display(i)
    display(df[i].unique())
    display(df[i].nunique())

'client_id'

array(['0xA095932790098744A2325A8D152C05C7',
       '0xF75BAA20A603A2479005DEB80464336E',
       '0x0D18081E0556AF4F95A931FBA4159D52', ...,
       '0xFAEF8B9104EA1F44972A461ED819B41F',
       '0xA635D01777FFE9F411EE92629EDE5758',
       '0xA635D01777FFE9F411EE89198C301878'], dtype=object)

198977

'npo_account_id'

array(['0xD4DBBAC6561929409BA45725A220613E',
       '0x3DCFF11974B3D8458CE3569DE9E855D5',
       '0x9DADF88CB3407C4E89403315F640393E', ...,
       '0x586E40A49367494CA8BF8F90D17F668F',
       '0xA635D01777FFE9F411EE93593F55A8EF',
       '0xA635D01777FFE9F411EE9D73DADDC2ED'], dtype=object)

...

array([nan, '2009-03-11', '2010-03-17', ..., '2015-11-25', '1996-05-16',
       '1994-04-02'], dtype=object)

6425

'region'

array(['САМАРСКАЯ ОБЛ', 'ЛИПЕЦКАЯ ОБЛ', 'ОРЛОВСКАЯ ОБЛ',
       'СВЕРДЛОВСКАЯ ОБЛ', 'НИЖЕГОРОДСКАЯ ОБЛ', 'КРАСНОЯРСКИЙ КРАЙ',
       'УДМУРТСКАЯ РЕСП', 'МОСКВА Г', 'ВОЛОГОДСКАЯ ОБЛ', 'ИРКУТСКАЯ ОБЛ',
       'ТУЛЬСКАЯ ОБЛ', 'НОВГОРОДСКАЯ ОБЛ', 'ЛЕНИНГРАДСКАЯ ОБЛ',
       'ОРЕНБУРГСКАЯ ОБЛ', 'АРХАНГЕЛЬСКАЯ ОБЛ', 'ЧЕЛЯБИНСКАЯ ОБЛ',
       'САХАЛИНСКАЯ ОБЛ', 'КРАСНОДАРСКИЙ КРАЙ',
       'ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - ЮГРА АО', 'МАРИЙ ЭЛ РЕСП',
       'ЧЕЧЕНСКАЯ РЕСП', nan, 'ХАНТЫ-МАНСИЙСКИЙ-ЮГРА АО',
       'САНКТ-ПЕТЕРБУРГ Г', 'БАШКОРТОСТАН РЕСП', 'БЕЛГОРОДСКАЯ ОБЛ',
       'УЛЬЯНОВСКАЯ ОБЛ', 'ВОРОНЕЖСКАЯ ОБЛ', 'САРАТОВСКАЯ ОБЛ',
       'СЕВЕРНАЯ ОСЕТИЯ - АЛАНИЯ РЕСП', 'МОСКОВСКАЯ ОБЛ', 'КАРЕЛИЯ',
       'ХАБАРОВСКИЙ КРАЙ', 'КУГАРЧИНСКИЙ', 'АЛТАЙСКИЙ КРАЙ',
       'КУРСКАЯ ОБЛ', 'ТЮМЕНСКАЯ ОБЛ', 'ТОМСКАЯ ОБЛ', 'ПРИМОРСКИЙ КРАЙ',
       'ВОЛГОГРАДСКАЯ ОБЛ', 'ЯРОСЛАВСКАЯ ОБЛ', 'КОМИ РЕСП',
       'КАЛМЫКИЯ РЕСП', 'АМУРСКАЯ ОБЛ', 'ОМСКАЯ ОБЛ', 'ВЛАДИМИРСКАЯ ОБЛ',
       'КАРЕЛИЯ РЕСП',

448

In [11]:
df = df.drop(['client_id', 'npo_account_id'], axis=1)

In [12]:
# df['frst_pmnt_date'] = [pd.to_datetime(date, format='%Y-%m-%d').timestamp() for date in df['frst_pmnt_date']]

In [13]:
# df['frst_pmnt_date'] = [pd.to_datetime(date, format='%Y-%m-%d').timestamp() for date in df['frst_pmnt_date']]
# converted_data1 = pd.to_datetime(df['frst_pmnt_date'])
# converted_data2 = pd.to_datetime(df['lst_pmnt_date_per_qrtr'])

# numeric_data1 = [date.timestamp() for date in converted_data1]
# numeric_data2 = [date.timestamp() for date in converted_data2]

In [14]:
from sklearn.preprocessing import LabelEncoder


In [15]:
df['region'] = LabelEncoder().fit_transform(df['region'])

In [16]:
df['quarters'] = [int(date[-1]) for date in df['quarter']]

In [17]:
df = df.drop(['quarter'], axis=1)

In [18]:
df = df.drop(['region', 'frst_pmnt_date', 'lst_pmnt_date_per_qrtr'], axis=1)

In [ ]:
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier, Pool
import matplotlib.pyplot as plt

# Разделение данных на тренировочный и валидационный наборы
X_train, X_val, y_train, y_val = train_test_split(df.drop(['churn'], axis=1), df['churn'],
                                                  test_size=0.2, random_state=42)

# Создание и обучение модели CatBoost с учетом дисбаланса классов и использованием GPU
model = CatBoostClassifier(iterations=500,
                   learning_rate=0.1,
                   depth=10,
                   loss_function='Logloss',
                   eval_metric='F1',  # Здесь меняем на F1
                   random_seed=0,
                   class_weights=[1, 10]
#                            ,  # примерное соотношение классов
#                    task_type='GPU'
                          )

# Обучение модели
model.fit(X_train, y_train)

# Получение предсказаний на валидационном наборе данных
val_preds = model.predict(X_val)

# Вычисление ROC-AUC на валидационном наборе данных
f1_score = f1_score(y_val, val_preds)
print("F1 на валидационном наборе данных:", f1_score)

# Отображение графика
plt.show()

0:	learn: 0.7753851	total: 717ms	remaining: 5m 57s
1:	learn: 0.7670957	total: 1.33s	remaining: 5m 30s
2:	learn: 0.7720927	total: 1.94s	remaining: 5m 20s
3:	learn: 0.7781418	total: 2.56s	remaining: 5m 17s
4:	learn: 0.7790295	total: 3.18s	remaining: 5m 15s
5:	learn: 0.7804328	total: 3.87s	remaining: 5m 18s
6:	learn: 0.7808582	total: 4.49s	remaining: 5m 16s
7:	learn: 0.7923087	total: 5.14s	remaining: 5m 15s
8:	learn: 0.7939195	total: 5.83s	remaining: 5m 18s
9:	learn: 0.7977752	total: 6.48s	remaining: 5m 17s
10:	learn: 0.7997457	total: 7.12s	remaining: 5m 16s
11:	learn: 0.8007218	total: 7.71s	remaining: 5m 13s
12:	learn: 0.8023962	total: 8.39s	remaining: 5m 14s
13:	learn: 0.8055635	total: 8.99s	remaining: 5m 12s
14:	learn: 0.8078594	total: 9.71s	remaining: 5m 14s
15:	learn: 0.8099798	total: 10.4s	remaining: 5m 13s
16:	learn: 0.8130909	total: 11s	remaining: 5m 13s


317:	learn: 0.9022699	total: 1m 11s	remaining: 41s
318:	learn: 0.9024117	total: 1m 11s	remaining: 40.7s
319:	learn: 0.9026659	total: 1m 12s	remaining: 40.5s
320:	learn: 0.9026896	total: 1m 12s	remaining: 40.3s
321:	learn: 0.9028349	total: 1m 12s	remaining: 40.1s
322:	learn: 0.9028989	total: 1m 12s	remaining: 39.9s
323:	learn: 0.9030197	total: 1m 12s	remaining: 39.6s
324:	learn: 0.9031010	total: 1m 13s	remaining: 39.4s
325:	learn: 0.9031944	total: 1m 13s	remaining: 39.2s
326:	learn: 0.9032969	total: 1m 13s	remaining: 39s
327:	learn: 0.9034710	total: 1m 13s	remaining: 38.8s
328:	learn: 0.9035692	total: 1m 14s	remaining: 38.5s
329:	learn: 0.9037081	total: 1m 14s	remaining: 38.3s
330:	learn: 0.9038618	total: 1m 14s	remaining: 38.1s
331:	learn: 0.9039717	total: 1m 14s	remaining: 37.9s
332:	learn: 0.9042107	total: 1m 15s	remaining: 37.6s
333:	learn: 0.9043232	total: 1m 15s	remaining: 37.4s
334:	learn: 0.9044839	total: 1m 15s	remaining: 37.2s
335:	learn: 0.9046149	total: 1m 15s	remaining: 37s

473:	learn: 0.9205094	total: 1m 47s	remaining: 5.91s
474:	learn: 0.9208767	total: 1m 47s	remaining: 5.68s
475:	learn: 0.9210370	total: 1m 48s	remaining: 5.46s
476:	learn: 0.9210245	total: 1m 48s	remaining: 5.23s
477:	learn: 0.9210619	total: 1m 48s	remaining: 5s
478:	learn: 0.9211406	total: 1m 48s	remaining: 4.78s
479:	learn: 0.9213522	total: 1m 49s	remaining: 4.55s
480:	learn: 0.9215474	total: 1m 49s	remaining: 4.32s
481:	learn: 0.9216464	total: 1m 49s	remaining: 4.09s
482:	learn: 0.9217568	total: 1m 49s	remaining: 3.87s
483:	learn: 0.9217874	total: 1m 50s	remaining: 3.64s
484:	learn: 0.9219497	total: 1m 50s	remaining: 3.41s
485:	learn: 0.9219998	total: 1m 50s	remaining: 3.18s
486:	learn: 0.9220988	total: 1m 50s	remaining: 2.96s
487:	learn: 0.9222830	total: 1m 50s	remaining: 2.73s
488:	learn: 0.9223837	total: 1m 51s	remaining: 2.5s
489:	learn: 0.9224161	total: 1m 51s	remaining: 2.27s
490:	learn: 0.9225523	total: 1m 51s	remaining: 2.05s
491:	learn: 0.9225526	total: 1m 51s	remaining: 1.8

In [ ]:
import numpy as np

In [ ]:
unique_values = np.unique(val_preds)

In [ ]:
unique_valuesc

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
from torch.utils.data import Dataset, TensorDataset
from sklearn import preprocessing


# Предположим, что у вас уже есть данные df, подготовленные для обучения модели

# Класс для создания датасета PyTorch
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
X_train, X_val, y_train, y_val = train_test_split(df.drop(['churn'], axis=1),
                                                  df['churn'], test_size=0.2, random_state=42)


In [ ]:
scaler = preprocessing.MinMaxScaler()
X_train = scaler.fit_transform(X_train.values)
X_val =  scaler.fit_transform(X_val.values)

In [ ]:
X_train.shape

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

# Define your neural network model
class Classifier(nn.Module):
    def __init__(self, input_size):
        super(Classifier, self).__init__()
        self.fc0 = nn.Linear(input_size, 24)
        self.fc1 = nn.Linear(24, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.5)  # Adding dropout for regularization

    def forward(self, x):
        x = torch.relu(self.fc0(x))
        x = self.dropout(x)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.sigmoid(self.fc3(x))
        return x
    
    def get_embed(self, x):
        x = torch.relu(self.fc0(x))
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return x
    
# Define your dataset
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

bs = 32768

# Move model and data to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Classifier(input_size=X_train.shape[1]).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_dataset = CustomDataset(X_train, y_train.values)
train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True)

val_dataset = CustomDataset(X_val, y_val.values)
val_loader = DataLoader(val_dataset, batch_size=bs)

In [ ]:
num_epochs = 10
train_losses = []


for epoch in range(num_epochs):
    embeddings_train = []
    y_train_for_emb = []
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        y_train_for_emb.extend(labels)
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        outputs = torch.sigmoid(outputs)
        loss = criterion(outputs, labels.unsqueeze(1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

        embeddings = model.get_embed(inputs)
        embeddings = [a.detach().cpu().numpy() for a in embeddings]

        embeddings_train.extend(embeddings)


    epoch_loss = running_loss / len(train_loader.dataset)
    train_losses.append(epoch_loss)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")
    # Evaluation


model.eval()
val_preds = []
val_labels = []
embeddings_test = []
y_test_for_emb = []

with torch.no_grad():
    for inputs, labels in val_loader:
        y_test_for_emb.extend(labels)
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        val_preds.extend(outputs.cpu().numpy())
        val_labels.extend(labels.cpu().numpy())

        embeddings = model.get_embed(inputs).cpu().numpy()
        embeddings_test.extend(embeddings)

# Calculate ROC AUC score
roc_auc = f1_score(val_labels, val_preds)
print(f"Validation F1: {roc_auc:.4f}")

# Plot training loss
plt.plot(train_losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.show()

,slctn_nmbr,npo_accnts_nmbr,pmnts_type,year,gender,age,clnt_cprtn_time_d,actv_prd_d,lst_pmnt_rcnc_d,balance,...,lk,assignee_npo,assignee_ops,postal_code,citizen,fact_addrss,appl_mrkr,evry_qrtr_pmnt,churn,quarters
0,1,1,2,2017,1,48,8091,0,6757,679.37,...,-1,-1,-1,446254.0,-1,-1,0,0,0,4
1,0,2,2,2009,1,35,3303,1531,34,36395.87,...,-1,-1,-1,399841.0,-1,1,0,1,0,1
2,2,1,1,2010,1,47,1163,1121,34,5569.85,...,-1,-1,-1,302038.0,1,1,0,1,0,1
3,0,1,1,2005,-1,42,47,0,0,81.37,...,-1,-1,-1,620034.0,1,1,0,1,0,4
4,0,2,2,2007,1,42,876,853,30,6937.65,...,-1,-1,-1,398035.0,1,1,0,1,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3589913,2,1,1,2012,1,40,3687,2320,28,42022.77,...,-1,-1,-1,162690.0,-1,1,0,0,0,2
3589914,0,1,3,2018,1,41,1204,1186,28,298535.72,...,1,-1,-1,628303.0,1,1,0,1,0,4
3589915,0,1,3,2016,1,32,474,456,23,39954.46,...,-1,-1,-1,628401.0,1,1,0,0,0,4
3589916,1,1,2,1999,-1,41,611,0,611,59.89,...,-1,-1,-1,249130.0,-1,-1,0,0,0,1


In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import catboost 

In [24]:
df = pd.read_csv('train.csv')

In [25]:
df.drop_duplicates(inplace = True)

In [26]:
# df['frst_pmnt_date'] = [pd.to_datetime(date, format='%Y-%m-%d').timestamp() for date in df['frst_pmnt_date']]

In [27]:
df.columns
# Нужно дропнуть категориальные колонки и оставить только одну, оставим frst_pmnt_date
df = df.drop(['quarter', 'region', 'lst_pmnt_date_per_qrtr', 'client_id', 'npo_account_id', 'churn', 'postal_code'], axis=1)

In [28]:
# data_with_missing = df[df.isnull()] #test 
# data_without_missing = df.dropna() # train 
# # data_without_missing.isna().sum()

In [29]:
data_with_missing = df[df['frst_pmnt_date'].isna()]
data_without_missing = df[df['frst_pmnt_date'].notna()]

In [30]:
converted_data1 = pd.to_datetime(data_without_missing['frst_pmnt_date'])
# converted_data2 = pd.to_datetime(df['lst_pmnt_date_per_qrtr'])
data_without_missing['numeric_data1'] = [date.timestamp() for date in converted_data1]
# numeric_data2 = [date.timestamp() for date in converted_data2]

C:\Users\MadBoss\AppData\Local\Temp\ipykernel_10428\1886787826.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_without_missing['numeric_data1'] = [date.timestamp() for date in converted_data1]


In [31]:
# data_without_missing["frst_pmnt_date"]
data_without_missing = data_without_missing.drop('frst_pmnt_date', axis=1)

In [32]:
X = data_without_missing.drop(columns=["numeric_data1"])
y = data_without_missing["numeric_data1"]

In [33]:
# X.select_dtypes(include=['object']) 

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [35]:
from catboost import CatBoostRegressor
model = CatBoostRegressor()
params = {'task_type': 'GPU',
          'loss_function': 'RMSE',
          'eval_metric':'MAPE',
          'early_stopping_rounds': 200,
          'verbose': 200,
          'random_seed': 1 
         }

model = CatBoostRegressor(**params)
model.fit(X_train, y_train, 
          eval_set=(X_test, y_test), 
          use_best_model=True, 
          plot=True
         );

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.144504
0:	learn: 0.1266291	test: 0.1265259	best: 0.1265259 (0)	total: 23.2ms	remaining: 23.2s
200:	learn: 0.0055277	test: 0.0055371	best: 0.0055371 (200)	total: 3.75s	remaining: 14.9s
400:	learn: 0.0043076	test: 0.0043192	best: 0.0043192 (400)	total: 7.47s	remaining: 11.2s
600:	learn: 0.0037523	test: 0.0037648	best: 0.0037648 (600)	total: 11.2s	remaining: 7.44s
800:	learn: 0.0034098	test: 0.0034238	best: 0.0034238 (800)	total: 14.8s	remaining: 3.69s
999:	learn: 0.0031617	test: 0.0031774	best: 0.0031774 (999)	total: 18.5s	remaining: 0us
bestTest = 0.003177427749
bestIteration = 999


In [42]:
predicts = model.predict(data_with_missing)
predicts

array([1.54596239e+09, 1.59092170e+09, 1.60850061e+09, ...,
       1.58794636e+09, 1.57695212e+09, 1.58522661e+09])